In [1]:
# ------------------------------------------------------------
# (a) Import required libraries
# ------------------------------------------------------------
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense


# ------------------------------------------------------------
# (b) Upload / access the dataset + preprocessing
# ------------------------------------------------------------
data = pd.read_csv('creditcard.csv')     # Load dataset

# Normalize the "Amount" feature
data['Amount'] = StandardScaler().fit_transform(data[['Amount']])

# Drop the "Time" column (not needed)
data = data.drop(['Time'], axis=1)

# Use only normal transactions (Class = 0) for training
# Autoencoder learns normal pattern, anomalies → high error
x_train = data[data['Class'] == 0].drop(['Class'], axis=1).values


# ------------------------------------------------------------
# (c) Encoder converts it into latent representation
# ------------------------------------------------------------
inp = Input((29,))                      # Input layer with 29 features
enc = Dense(16, activation='relu')(inp)  # Encoder layer
lat = Dense(8, activation='relu')(enc)   # Latent (compressed) representation


# ------------------------------------------------------------
# (d) Decoder networks convert it back to original input
# ------------------------------------------------------------
dec = Dense(16, activation='relu')(lat)   # Decoder layer
out = Dense(29, activation='linear')(dec) # Output layer reconstructing input

# Build the complete Autoencoder model
model = Model(inp, out)


# ------------------------------------------------------------
# (e) Compile the model with Optimizer, Loss, Evaluation Metrics
# ------------------------------------------------------------
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


# Train the autoencoder
# Training input = output, since goal is reconstruction
model.fit(x_train, x_train, epochs=5, batch_size=32)


Epoch 1/5
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.4268 - mae: 0.4150 
Epoch 2/5
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 0.3255 - mae: 0.3535
Epoch 3/5
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 0.3085 - mae: 0.3399
Epoch 4/5
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 0.2981 - mae: 0.3361  
Epoch 5/5
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.2884 - mae: 0.3320
